## **Federated Learning for attack detection: 5 nodes sharing gradients: GEOMETRIC MEDIAN**

IDs from this file = **idGEOxy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [15]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')
test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_15184/3457440085.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_15184/3457440085.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')


In [16]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)
xcomplete, ycomplete = preprocessing(test_complete)

### Experiments with datasets

#### Dataset Filt5A

In [17]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part5.csv', low_memory=False)

In [18]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [19]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idGEO00.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [20]:
def weiszfeld_update(points, tol=1e-5, max_iter=100):
    """
    Compute the geometric median using Weiszfeld's algorithm.

    Args:
    points: A list of points (numpy arrays) to find the geometric median of.
    tol: Tolerance for stopping criterion.
    max_iter: Maximum number of iterations.

    Returns:
    The geometric median of the points.
    """
    points = np.array(points)
    median = np.mean(points, axis=0)

    for _ in range(max_iter):
        distances = np.linalg.norm(points - median, axis=1)
        nonzero_distances = np.where(distances != 0, distances, np.finfo(float).eps)
        weights = 1 / nonzero_distances
        new_median = np.sum(points * weights[:, None], axis=0) / np.sum(weights)

        if np.linalg.norm(new_median - median) < tol:
            return new_median

        median = new_median

    return median

def aggregate(grad_list):
    """
    Apply Geometric Median aggregation method to a list of gradients.

    Args:
    grad_list: List of gradients from different models. Each element in the list
               is a list of gradients for each layer of a model.

    Returns:
    The geometric median of the gradients.
    """
    # Flatten gradients to compute geometric median
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute geometric median using Weiszfeld's algorithm
    flat_median = weiszfeld_update(flat_grads)

    # Reshape the flat median back to the original shape
    median_grad = []
    index = 0
    for grad in grad_list[0]:
        shape = tf.shape(grad)
        size = tf.reduce_prod(shape)
        median_grad.append(tf.reshape(flat_median[index:index + size], shape))
        index += size

    return median_grad

In [21]:
global_model = build_model((24,1,1))

In [22]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [23]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [24]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idGEO00.hdf5')

Epoch 1/5
31/31 [==============================] - 6s 184ms/step - loss: 0.3827 - accuracy: 0.9650 - val_loss: 1.8270 - val_accuracy: 0.6196
Epoch 2/5
31/31 [==============================] - 6s 187ms/step - loss: 0.0620 - accuracy: 0.9916 - val_loss: 0.9786 - val_accuracy: 0.6200
Epoch 3/5
31/31 [==============================] - 6s 204ms/step - loss: 0.0299 - accuracy: 0.9919 - val_loss: 0.6993 - val_accuracy: 0.6301
Epoch 4/5
31/31 [==============================] - 6s 193ms/step - loss: 0.0268 - accuracy: 0.9922 - val_loss: 0.5889 - val_accuracy: 0.6538
Epoch 5/5
31/31 [==============================] - 6s 200ms/step - loss: 0.0253 - accuracy: 0.9920 - val_loss: 0.5823 - val_accuracy: 0.6533
Epoch 1/5
31/31 [==============================] - 10s 320ms/step - loss: 0.0391 - accuracy: 0.9900 - val_loss: 0.5308 - val_accuracy: 0.7069
Epoch 2/5
31/31 [==============================] - 9s 293ms/step - loss: 0.0257 - accuracy: 0.9913 - val_loss: 0.4643 - val_accuracy: 0.6890
Epoch 3/5
31

In [25]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.05698449909687042, 0.2462124079465866, 0.0956316888332367], [0.05720860883593559, 0.33429884910583496, 0.10186640918254852], [0.05987226217985153, 0.5076313018798828, 0.11560719460248947]]
Accuracy for iterations:  [[0.971055269241333, 0.8905217051506042, 0.9520816802978516], [0.97443687915802, 0.8481798768043518, 0.95700603723526], [0.9754886031150818, 0.7823330760002136, 0.9564061760902405]]
F1 for iterations:  [[0.9710081194459265, 0.8882009653299612, 0.952007564857771], [0.9744031721520753, 0.8422994633806594, 0.9569546411223356], [0.9754574277724276, 0.7656067274071545, 0.9563495550864797]]
Precision for iterations:  [[0.9717991720361379, 0.9029758527340818, 0.9548915693299354], [0.9749692369443568, 0.8739761527340623, 0.9591991495473647], [0.9759929096074956, 0.8355189360247318, 0.9587870328222783]]
Recall for iterations:  [[0.9710552455520173, 0.8905216885853375, 0.9520816913000718], [0.974436881007333, 0.8481799026375063, 0.957006047332408], [0.9754886

#### Dataset Filt5B

In [26]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part5.csv', low_memory=False)

In [27]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [28]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idGEO01.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [29]:
global_model = build_model((24,1,1))

In [30]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [31]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [32]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idGEO01.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 148ms/step - loss: 0.3854 - accuracy: 0.9307 - val_loss: 2.1654 - val_accuracy: 0.5486
Epoch 2/5
31/31 [==============================] - 5s 159ms/step - loss: 0.0713 - accuracy: 0.9910 - val_loss: 1.5919 - val_accuracy: 0.5526
Epoch 3/5
31/31 [==============================] - 5s 164ms/step - loss: 0.0307 - accuracy: 0.9919 - val_loss: 0.7599 - val_accuracy: 0.5617
Epoch 4/5
31/31 [==============================] - 5s 148ms/step - loss: 0.0255 - accuracy: 0.9919 - val_loss: 0.6106 - val_accuracy: 0.6172
Epoch 5/5
31/31 [==============================] - 5s 147ms/step - loss: 0.0241 - accuracy: 0.9921 - val_loss: 0.6820 - val_accuracy: 0.5831
Epoch 1/5
31/31 [==============================] - 4s 145ms/step - loss: 0.0311 - accuracy: 0.9904 - val_loss: 0.4304 - val_accuracy: 0.7339
Epoch 2/5
31/31 [==============================] - 5s 150ms/step - loss: 0.0232 - accuracy: 0.9918 - val_loss: 0.4138 - val_accuracy: 0.7199
Epoch 3/5
31/

In [33]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.13843773305416107, 0.2994016706943512, 0.19816967844963074], [0.09693670272827148, 0.22727085649967194, 0.15739957988262177], [0.09774153679609299, 0.19225603342056274, 0.15302471816539764]]
Accuracy for iterations:  [[0.9239533543586731, 0.8046937584877014, 0.8910852670669556], [0.9459522366523743, 0.886380136013031, 0.9138447046279907], [0.9443600177764893, 0.9175688624382019, 0.9135656952857971]]
F1 for iterations:  [[0.923226195238611, 0.7930928482415546, 0.8898655570024935], [0.9456611518452166, 0.8838644354346318, 0.9132670204316425], [0.9440509381249023, 0.9165176387721264, 0.9129895350603425]]
Precision for iterations:  [[0.9325484311301909, 0.844653757561124, 0.9092111666571719], [0.9501438442304992, 0.8994442471374056, 0.9251708336730285], [0.948714588262095, 0.9238694111759653, 0.9248167226582382]]
Recall for iterations:  [[0.9239533728709574, 0.8046937440964906, 0.8910852415794209], [0.9459522627012182, 0.8863801496766693, 0.9138446944597507], [0.9

#### Dataset Filt5C

In [34]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part5.csv', low_memory=False)

In [35]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [36]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idGEO02.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [37]:
global_model = build_model((24,1,1))

In [38]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [39]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [40]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idGEO02.hdf5')

Epoch 1/5
16/16 [==============================] - 11s 709ms/step - loss: 0.5450 - accuracy: 0.7213 - val_loss: 1.6593 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 9s 591ms/step - loss: 0.2015 - accuracy: 0.9510 - val_loss: 3.3367 - val_accuracy: 0.1334
Epoch 3/5
16/16 [==============================] - 8s 473ms/step - loss: 0.0783 - accuracy: 0.9893 - val_loss: 3.4470 - val_accuracy: 0.1342
Epoch 4/5
16/16 [==============================] - 8s 503ms/step - loss: 0.0497 - accuracy: 0.9893 - val_loss: 1.8136 - val_accuracy: 0.1379
Epoch 5/5
16/16 [==============================] - 10s 640ms/step - loss: 0.0370 - accuracy: 0.9898 - val_loss: 1.4136 - val_accuracy: 0.1942
Epoch 1/5
14/14 [==============================] - 9s 626ms/step - loss: 0.0227 - accuracy: 0.9948 - val_loss: 0.8248 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 5s 364ms/step - loss: 0.0214 - accuracy: 0.9950 - val_loss: 0.6812 - val_accuracy: 0.7284
Epoch 3

In [41]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08878658711910248, 0.17068345844745636, 0.10156106948852539], [0.05471533536911011, 0.13642050325870514, 0.05986447632312775], [0.05124828219413757, 0.08169283717870712, 0.04878015071153641]]
Accuracy for iterations:  [[0.9864369630813599, 0.9422727823257446, 0.9779450297355652], [0.9896140694618225, 0.9542069435119629, 0.9861197471618652], [0.9898843169212341, 0.9862130284309387, 0.9903466105461121]]
F1 for iterations:  [[0.9864399692915574, 0.9419277626643375, 0.9779443146183981], [0.9896183584435108, 0.954072465228889, 0.9861195645109446], [0.9898891213014196, 0.9862322162185762, 0.990345722631171]]
Precision for iterations:  [[0.9864773933002633, 0.9442268026446218, 0.9780063543909912], [0.989761533913747, 0.9547431539113531, 0.9861433849071188], [0.9900909806559807, 0.9865685957771482, 0.9905226280682651]]
Recall for iterations:  [[0.9864369394373192, 0.9422727602993534, 0.9779450229826532], [0.9896140699406936, 0.9542069316282787, 0.9861197329966729], [0